In [4]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import numpy as np
from pyspark.mllib.linalg.distributed import *
from pyspark.mllib.linalg import *
from scipy.sparse.linalg import *
from scipy.sparse import *
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [5]:
icm = sc.textFile("../icm.csv")
icm = icm.map(lambda l: l.split(','))\
            .filter(lambda line: line[0] != 'itemId')\
            .map(lambda x: (int(x[0]), int(x[1]), 1))
trainSet = sc.textFile("../train.csv")
trainSet = trainSet.map(lambda l: l.split(','))
trainSet = trainSet.filter(lambda line: 'userId' not in line)
trainSet = trainSet.map(lambda line: (int(line[0]), int(line[1]), int(line[2])))
#FOR THE TOP POP
itemSet = trainSet.map(lambda x: (x[1], x[2]))
itemsCount = trainSet.map(lambda x: (x[1],1)).reduceByKey(lambda x,y : x + y)
itemsCount_dict = itemsCount.collectAsMap()
#-----------------------------------------
featureFreq = icm.map(lambda x: (x[1],1)).reduceByKey(lambda x, y: x + y)
featureFreqDict = featureFreq.collectAsMap()
prodCount= icm.map(lambda x: x[0]).distinct().count()
featureIdf = featureFreq.map(lambda x: (x[0],np.log10(prodCount/x[1])))
featureIdfDict = featureIdf.collectAsMap()

targetUsers = sc.textFile("../target_user.csv").filter(lambda x: "userId" not in x).map(lambda x: int(x))
targets=targetUsers.collect()

######
#TEST COMPUTING THE PREDICTION NORMALIZING BY THE N OF FEATURES AND NOT THE SQRT OF IT
norms = icm.map(lambda x: (x[0],1))\
                .reduceByKey(lambda x, y: x+y).mapValues(lambda x: np.sqrt(x))\
                .collectAsMap()

normalized = icm.map(lambda x: (x[0], x[1], x[2]/norms[x[0]]))

In [6]:
len(targets)

4196

In [7]:
#Just consider the row of the users to predict
#IF you0re asking, only the idf depends on all the training set, in fact it is computed before reducing the trainset
trainSet=trainSet.filter(lambda x: x[0] in targets)
print(trainSet.count())
data = trainSet.map(lambda x: x[2]).collect()
rows = trainSet.map(lambda x: x[0]).collect()
cols = trainSet.map(lambda x: x[1]).collect()
data.append(0)
rows.append(15364)
cols.append(37142)
userItem=csr_matrix((data,(rows,cols)))
print("userItem shape:",userItem.shape)
data = normalized.map(lambda x: x[2]).collect()
rows = normalized.map(lambda x: x[0]).collect()
cols = normalized.map(lambda x: x[1]).collect()
data.append(0)
rows.append(37142)
cols.append(80)
itemFeature = csc_matrix((data,(rows,cols)))
print("itemFeat shape:",itemFeature.shape)

46750
userItem shape: (15365, 37143)
itemFeat shape: (37143, 19716)


In [20]:
u, s, vt = svds(itemFeature,k=50, which="LM")
print("Shape of U: ", u.shape)
print("Shape of s: ", s.shape)
print("Shape of Vt: ", vt.shape)

Shape of U:  (37143, 50)
Shape of s:  (50,)
Shape of Vt:  (50, 19716)


In [31]:
print(s)

print(type(u))

[  3.42873494   3.43227183   3.45570265   3.46486617   3.471087
   3.47976741   3.50129029   3.50465823   3.51832771   3.52656382
   3.54105666   3.55621432   3.57632724   3.58495768   3.61700315
   3.63115945   3.63791877   3.65201843   3.65794517   3.68578184
   3.70828752   3.76955018   3.82643947   3.91094089   3.9476637    4.15166
   4.20664377   4.30718711   4.52396193   8.43965313   9.1006113
  11.41398587  11.53827133  12.61509155  13.70147779  14.5415358
  14.69673431  15.37852119  16.05272209  17.45456075  19.37911124
  20.11457575  22.84986982  26.64749568  29.52118109  29.88638904
  33.90362104  39.45063873  47.37983098  70.57636104]
<class 'numpy.ndarray'>


In [22]:
print(u)

[[  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  3.28775749e-04  -8.20843137e-05   2.35000614e-04 ...,  -5.70611914e-04
    2.42708694e-03   5.62752102e-03]
 [ -2.13400583e-04  -1.79382564e-03  -2.32926927e-03 ...,  -2.11428694e-03
    1.37908509e-03   6.63503272e-03]
 ..., 
 [ -6.44592123e-05  -2.02078323e-05  -5.26308887e-04 ...,  -1.35726033e-03
   -9.55115958e-03   2.23938369e-03]
 [ -8.80653108e-03  -1.34621384e-02  -7.91347921e-03 ...,  -2.96242110e-03
    1.84362327e-03   6.31954228e-03]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]


In [32]:
userFeature = userItem.dot(u)
print(userFeature.shape)

(15365, 50)
<class 'scipy.sparse.csr.csr_matrix'>


In [33]:
prediction = userFeature.dot(u.transpose())
prediction.shape

prediction[4]

array([ 0.        , -0.00034627,  0.00163146, ..., -0.00024361,
        0.00179577,  0.        ])

In [30]:
type(prediction)

numpy.ndarray

In [34]:
numberOfRecommendations=5
#TOP POPULAR
cost=8
avgRatings=itemSet.reduceByKey(lambda x,y: x+y)
avgRatings=avgRatings.map(lambda x: (x[0],x[1]/(itemsCount_dict[x[0]]+cost)))
avgRatings.take(5)
itemOrderByPop=avgRatings.sortBy(lambda x: x[1], ascending=False)
itemPop = np.array(itemOrderByPop.map(lambda x: x[0]).collect())
seenItems= trainSet.map(lambda x: (x[0],[x[1]])).reduceByKey(lambda x,y: x + y)
seenItemsDict=seenItems.collectAsMap()
#--------------------------------------------------------------

def recommendTopPop(user_id, removeSeen=True):
    seenItems = np.array(seenItemsDict[user_id])
    recommendedList = itemPop
    if(removeSeen):
        unseen_mask = np.in1d(recommendedList, seenItems, invert=True)
        recommendedList = recommendedList[unseen_mask]       
    return recommendedList[:numberOfRecommendations]


def fillWithTopPop(recommended,user):
    TopPop=recommendTopPop(user)
    for i in range (numberOfRecommendations-len(recommended)):
        recommended.append(TopPop[i])
    return recommended


def getRecommended(user):
    recommended = []
    itemsPred = prediction[user]

    for i in range(0,len(itemsPred)):
        if(itemsPred[i]!=0):
                if i not in seenItemsDict[user]:
                    recommended.append((i, itemsPred[i]))
    recommended.sort(key = lambda x: -x[1])
    recommended=recommended[:numberOfRecommendations]
    print(recommended)
    recommendedItems = list(map(lambda x: x[0], recommended))
    if(len(recommendedItems)<numberOfRecommendations):
        recommendedItems=fillWithTopPop(recommendedItems, user)
    return recommendedItems    



In [35]:
print("After function:")
print(getRecommended(100))
getRecommended(151)

After function:
[(2038, 0.022817463176642391), (33387, 0.021820938379803447), (29413, 0.017846384388044385), (33143, 0.016661693898902822), (36535, 0.016661693898902822)]
[2038, 33387, 29413, 33143, 36535]
[(3768, 0.012495628201107136), (12841, 0.012114235142393848), (8784, 0.012037428922371779), (16899, 0.011952323170351029), (24376, 0.01189207444792046)]


[3768, 12841, 8784, 16899, 24376]

In [36]:
f=open("predictionsSVD.csv",'w')
i=0
for user in targets:
    print(user)
    f.write("{}".format(user)+',')
    rec=getRecommended(user)
    print(rec)
    for prod in rec:
        f.write(str(prod) +' ')
    f.write('\n')
    i=i+1
    print(i,"of", len(targets), "written")
f.close()

4
[(16451, 0.12086010620951755), (652, 0.11570690302690964), (28028, 0.11415075924873579), (8077, 0.11252997910661741), (26713, 0.11012080338930727)]
[16451, 652, 28028, 8077, 26713]
1 of 4196 written
5
[(6321, 0.052049876271963696), (7166, 0.052049876271963696), (24699, 0.052049876271963696), (1060, 0.051375577509534587), (32202, 0.051375577509534587)]
[6321, 7166, 24699, 1060, 32202]
2 of 4196 written
8
[(30677, 0.19747875530549147), (17821, 0.1837657213821115), (31730, 0.15512455045466603), (18558, 0.14375653585757886), (13042, 0.14250440673582693)]
[30677, 17821, 31730, 18558, 13042]
3 of 4196 written
9
[(2613, 0.018737036594793768), (5465, 0.018737036594793768), (6096, 0.018737036594793768), (20195, 0.018737036594793768), (20558, 0.018737036594793768)]
[2613, 5465, 6096, 20195, 20558]
4 of 4196 written
13
[(4850, 0.0067809399987908223), (5086, 0.0067809399987908223), (5526, 0.0067809399987908223), (10176, 0.0067809399987908223), (11745, 0.0067809399987908223)]
[4850, 5086, 5526, 1

61 of 4196 written
187
[(16845, 0.032618669460143641), (24376, 0.026076084205839303), (22003, 0.025827455538147158), (29403, 0.024549008557137337), (2143, 0.023643480909952098)]
[16845, 24376, 22003, 29403, 2143]
62 of 4196 written
189
[(36546, 0.12660512556020484), (27597, 0.12541633268885877), (32968, 0.11795528721318942), (9986, 0.11751942531546621), (28762, 0.11693580511725106)]
[36546, 27597, 32968, 9986, 28762]
63 of 4196 written
190
[(7019, 0.004818942906242068), (5866, 0.0047165602083442566), (21796, 0.0047165602083442566), (19243, 0.0046053162334606492), (4044, 0.0045169020868568001)]
[7019, 5866, 21796, 19243, 4044]
64 of 4196 written
191
[(26436, 0.013581643319925361), (31711, 0.013581643319925361), (16313, 0.01025692890014528), (5593, 0.0097159554005618928), (21934, 0.0096594831895556784)]
[26436, 31711, 16313, 5593, 21934]
65 of 4196 written
197
[(1464, 0.0643222473308947), (1772, 0.0643222473308947), (4561, 0.0643222473308947), (4619, 0.0643222473308947), (5546, 0.0643222

107 of 4196 written
359
[(33198, 1.177372711889777), (17759, 1.0475024283693029), (33757, 1.0343343293213529), (31129, 1.011306051598075), (5411, 0.98216710620683889)]
[33198, 17759, 33757, 31129, 5411]
108 of 4196 written
361
[(2144, 0.0091011813248266359), (7689, 0.0091011813248266359), (29019, 0.0091011813248266359), (33214, 0.0091011813248266359), (5434, 0.0090635000972913576)]
[2144, 7689, 29019, 33214, 5434]
109 of 4196 written
364
[(11124, 0.019553841212209804), (18479, 0.019553841212209804), (36547, 0.019553841212209804), (7961, 0.015039366650056368), (16141, 0.014891088179547275)]
[11124, 18479, 36547, 7961, 16141]
110 of 4196 written
365
[(16451, 0.3410953689757345), (2049, 0.33703947046663968), (652, 0.33051916856042413), (28028, 0.33005642634278748), (1306, 0.32845635038470217)]
[16451, 2049, 652, 28028, 1306]
111 of 4196 written
374
[(5671, 0.14619374383094411), (12370, 0.11106862191950022), (31929, 0.10992755829191382), (28687, 0.10862190237757995), (25560, 0.105950204259

[16845, 8784, 17475, 2143, 652]
188 of 4196 written
654
[(29992, 0.0028143443880946947), (16489, 0.0027758701818682615), (35673, 0.0027758701818682615), (1455, 0.0027121566930145046), (1967, 0.0027121566930145046)]
[29992, 16489, 35673, 1455, 1967]
189 of 4196 written
658
[(2613, 0.02299611043297), (5465, 0.02299611043297), (6096, 0.02299611043297), (20195, 0.02299611043297), (20558, 0.02299611043297)]
[2613, 5465, 6096, 20195, 20558]
190 of 4196 written
659
[(23588, 0.0089446777763134179), (27178, 0.0087588065761462306), (19115, 0.0086516290453639306), (20539, 0.0085687829004143304), (30638, 0.0082381108939884647)]
[23588, 27178, 19115, 20539, 30638]
191 of 4196 written
663
[(3995, 0.059271203469129889), (9867, 0.059271203469129889), (17843, 0.059271203469129889), (20801, 0.058477458514272398), (20922, 0.058477458514272398)]
[3995, 9867, 17843, 20801, 20922]
192 of 4196 written
665
[(16845, 0.002260873676330277), (1306, 0.0022205054899296669), (2049, 0.0020896872590452724), (16451, 0.

[18705, 26881, 2460, 8147, 21500]
299 of 4196 written
1043
[(2038, 6.2307789854791569e-05), (28522, 4.5860334223613898e-05), (6127, 4.4753310609331194e-05), (8106, 4.4753310609331194e-05), (14359, 4.4753310609331194e-05)]
[2038, 28522, 6127, 8106, 14359]
300 of 4196 written
1044
[(1346, 0.023953510756726044), (5792, 0.023953510756726044), (9970, 0.022754607754360688), (14013, 0.021390307766729909), (16920, 0.021379335764221751)]
[1346, 5792, 9970, 14013, 16920]
301 of 4196 written
1046
[(5434, 0.0083347345844007343), (5522, 0.0083347345844007343), (8516, 0.0083347345844007343), (12510, 0.0083347345844007343), (13698, 0.0083347345844007343)]
[5434, 5522, 8516, 12510, 13698]
302 of 4196 written
1055
[(16845, 0.069900648632883491), (9893, 0.069454843770394403), (2143, 0.063197907174185636), (31833, 0.058239123689727404), (24376, 0.058164625074208769)]
[16845, 9893, 2143, 31833, 24376]
303 of 4196 written
1060
[(18705, 0.030378892693454316), (2460, 0.028165194776280534), (8147, 0.027657286

...

3687 of 4196 written
13493
[(20801, 0.072074154896988846), (20922, 0.072074154896988846), (35227, 0.072074154896988846), (35564, 0.072074154896988846), (3995, 0.069631054249650132)]
[20801, 20922, 35227, 35564, 3995]
3688 of 4196 written
13495
[(30785, 0.20764603997291844), (31542, 0.20764603997291844), (13027, 0.18956793895660612), (6614, 0.18735086261204831), (14059, 0.17972687822709346)]
[30785, 31542, 13027, 6614, 14059]
3689 of 4196 written
13505
[(16451, 0.048066752072807556), (28028, 0.047192653282403914), (23949, 0.04684598322436339), (2041, 0.04676321733505303), (7961, 0.046528850051547235)]
[16451, 28028, 23949, 2041, 7961]
3690 of 4196 written
13512
[(9160, 0.31202538791386691), (9223, 0.25536219819051403), (28790, 0.24988888735084644), (36503, 0.24687123993410173), (15814, 0.23931968762121064)]
[9160, 9223, 28790, 36503, 15814]
3691 of 4196 written
13513
[(18314, 0.17953107663821632), (33897, 0.17124257657890174), (3203, 0.16663800689962158), (25217, 0.16547866478191028), (

[(1060, 0.035077934104472124), (32202, 0.035077934104472124), (6321, 0.03405545052350574), (7166, 0.03405545052350574), (24699, 0.03405545052350574)]
[1060, 32202, 6321, 7166, 24699]
3796 of 4196 written
13961
[(16845, 0.0093641241018775582), (5434, 0.0091726479904860516), (5522, 0.0091726479904860516), (8516, 0.0091726479904860516), (12510, 0.0091726479904860516)]
[16845, 5434, 5522, 8516, 12510]
3797 of 4196 written
13963
[(24376, 0.0024886848341910098), (4371, 0.0023883290952913263), (5299, 0.0023883290952913263), (6550, 0.0023883290952913263), (15766, 0.0023883290952913263)]
[24376, 4371, 5299, 6550, 15766]
3798 of 4196 written
13964
[(2287, 0.020097036640579897), (33865, 0.014160863102227502), (18090, 0.013347400436394262), (12236, 0.011887994342946072), (27569, 0.011256939144633228)]
[2287, 33865, 18090, 12236, 27569]
3799 of 4196 written
13969
[(1346, 0.024196267710482069), (5792, 0.024196267710482069), (1307, 0.023801823676455967), (4529, 0.021071293145668159), (24363, 0.018996

[(30677, 0.13181979852909678), (13042, 0.12739888175537289), (3482, 0.12054364921381069), (33037, 0.11566699174066035), (34458, 0.11448714626060086)]
[30677, 13042, 3482, 33037, 34458]
3871 of 4196 written
14243
[(12635, 0.031123468848839906), (6951, 0.02994033204500857), (24306, 0.029404770796458254), (822, 0.029089186049367617), (10565, 0.028859153509724064)]
[12635, 6951, 24306, 822, 10565]
3872 of 4196 written
14244
[(16451, 0.018064079947048388), (2049, 0.016008777294076289), (13726, 0.015793785123802355), (307, 0.015494269180850435), (1306, 0.015320237917887673)]
[16451, 2049, 13726, 307, 1306]
3873 of 4196 written
14245
[(7660, 0.020046505542599052), (15627, 0.020046505542599052), (25852, 0.020046505542599052), (26151, 0.020046505542599052), (26423, 0.020046505542599052)]
[7660, 15627, 25852, 26151, 26423]
3874 of 4196 written
14252
[(1346, 0.014106759082469346), (5792, 0.014106759082469346), (6814, 0.010129166681735), (8032, 0.010129166681735), (17417, 0.010129166681735)]
[1346

14535
[(1346, 0.032134935489447285), (5792, 0.032134935489447285), (9970, 0.027343188392054057), (1161, 0.02563730178053223), (18699, 0.02563730178053223)]
[1346, 5792, 9970, 1161, 18699]
3949 of 4196 written
14536
[(4951, 0.0039881427666155749), (6759, 0.0039881427666155749), (21169, 0.0039881427666155749), (1118, 0.0032592239186820682), (2196, 0.0032592239186820682)]
[4951, 6759, 21169, 1118, 2196]
3950 of 4196 written
14537
[(33198, 0.026746405255206364), (17759, 0.024789001124952737), (33757, 0.02396847760161766), (31129, 0.023678864607515605), (14885, 0.023614020200109817)]
[33198, 17759, 33757, 31129, 14885]
3951 of 4196 written
14541
[(13486, 0.12889585812827023), (19661, 0.1255531839521381), (33897, 0.1247904788454874), (15275, 0.12411824303498487), (25217, 0.12032161536651399)]
[13486, 19661, 33897, 15275, 25217]
3952 of 4196 written
14549
[(19345, 0.71895408551338746), (1306, 0.69596041062478942), (13325, 0.69103376863055843), (420, 0.68553617662961508), (16451, 0.68049277265

4029 of 4196 written
14785
[(35464, 0.032390673176595032), (1060, 0.030719402302667662), (32202, 0.030719402302667662), (6321, 0.030334888588185716), (7166, 0.030334888588185716)]
[35464, 1060, 32202, 6321, 7166]
4030 of 4196 written
14787
[(26436, 0.005449586168086819), (31711, 0.005449586168086819), (16313, 0.0047926263998181277), (17067, 0.0044028594415594898), (24032, 0.0042626387473634888)]
[26436, 31711, 16313, 17067, 24032]
4031 of 4196 written
14788
[(16845, 0.026155030097207307), (2143, 0.022763080704749396), (24376, 0.02138171492881091), (22003, 0.020084555152488238), (31833, 0.019375470549584342)]
[16845, 2143, 24376, 22003, 31833]
4032 of 4196 written
14789
[(16845, 0.090635406223892906), (22003, 0.071802313669011048), (31833, 0.071295302909379812), (24376, 0.071190667834139618), (9933, 0.067644586133403614)]
[16845, 22003, 31833, 24376, 9933]
4033 of 4196 written
14792
[(19279, 0.0098000340862222229), (4037, 0.0086094901377306698), (36748, 0.0086094901377306698), (3952, 0.

In [18]:
print("Result of getRecommended:\n",getRecommended(151))
print("Top 5 from the matrix: " ,prediction.getrow(151).toarray()[0])

[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
Result of getRecommended:
 [8013, 30796, 12535, 4223, 11554]
Top 5 from the matrix:  [ 0.          3.52206684  6.39351585 ...,  0.58138567  4.65589395  0.        ]


In [19]:
print("Result of getRecommended:\n",getRecommended(100))
print("Top 5 from the matrix: " ,prediction.getrow(100).toarray()[0])

[(12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616), (20532, 13.00817618709717), (19267, 12.695308889326265)]
Result of getRecommended:
 [12535, 4223, 11554, 20532, 19267]
Top 5 from the matrix:  [ 0.          2.36576972  3.58037592 ...,  0.          1.72884326  0.        ]
